# 🚀 Messages 이해하기

## 📚 학습 목표
1. LangChain의 Message 타입 이해하기
2. SystemMessage, HumanMessage, AIMessage의 역할과 차이점 파악
3. Message를 활용한 대화 흐름 관리 방법 학습
4. Message 구조와 메타데이터 활용 방법 이해

## 🎯 Messages란?
LangChain에서 Message는 LLM과의 대화를 구성하는 기본 단위입니다. 각 Message는 특정 역할(System, Human, AI)을 가지며, 대화의 맥락과 방향을 결정하는 중요한 요소입니다.

### 💡 주요 Message 타입
1. **SystemMessage**: AI의 역할과 행동 지침을 정의
2. **HumanMessage**: 사용자의 입력을 표현
3. **AIMessage**: AI의 응답을 표현
4. **ToolMessage**: 도구 실행 결과를 표현

### 🔄 Message의 특징
- **구조화된 형식**: 각 Message는 content와 추가 메타데이터를 포함
- **역할 기반**: 각 Message 타입은 특정 역할을 수행
- **컨텍스트 유지**: 대화의 맥락을 유지하고 전달
- **확장 가능**: 추가 메타데이터를 통해 기능 확장 가능

### 🔑 OpenAI Key 설정하기

In [1]:
import dotenv
import os

dotenv.load_dotenv(".env")
if not "OPENAI_API_KEY" in os.environ:
    raise Exception("OPENAI_API_KEY가 환경변수에 존재하지 않습니다.")

 ········


## 실습: 기본 Message 사용하기

### 1. 기본적인 Message 구성과 실행

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from pprint import pprint
import json

# OpenAI Chat 모델 인스턴스 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 메시지 정의
messages = [
    SystemMessage(content="당신은 심심풀이용 챗봇입니다. 상당히 밝고 친절합니다."),
    HumanMessage(content="안녕 만나서 반가워")
]

# 모델 실행
response = llm.invoke(messages)

# JSON 형식으로 보기 좋게 출력
formatted_content = json.dumps(response.model_dump(), indent=2, ensure_ascii=False)

print("\n📝 [입력 메시지]")
pprint(messages)

print("\n🔹 [Raw 응답 출력 - AIMessage]")
pprint(response)


📝 [입력 메시지]
[SystemMessage(content='당신은 심심풀이용 챗봇입니다. 상당히 밝고 친절합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕 만나서 반가워', additional_kwargs={}, response_metadata={})]

🔹 [Raw 응답 출력 - AIMessage]
AIMessage(content='안녕하세요! 만나서 반가워요! 오늘 기분은 어때요? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 40, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-d30711d9-043e-47cd-aeb4-06408b0bde1b-0', usage_metadata={'input_tokens': 40, 'output_tokens': 20, 'total_tokens': 60, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


### 📌 AIMessage 분석하기

`AIMessage`는 `response_metadata`, `usage_metadata`, `additional_kwargs`, `tool_calls` 등의 인자를 포함하며, **LLM 서비스 및 Agent 관점에서 가장 핵심적인 메시지 계층**입니다.  
아래 표를 통해 `AIMessage`의 주요 필드와 역할을 정리합니다.

---

#### **1. `response_metadata` 필드**
| **필드명**            | **설명** |
|----------------------|---------|
| `token_usage`        | LLM이 사용한 토큰 수 (입력/출력/총합) |
| `completion_tokens`  | 모델이 생성한 토큰 개수 |
| `prompt_tokens`      | 입력 프롬프트에 사용된 토큰 개수 |
| `total_tokens`       | 입력 + 출력 합계 |
| `model_name`         | 사용된 모델 버전 정보 |
| `system_fingerprint` | 모델의 내부 시스템 버전 관리용 식별자 |
| `finish_reason`      | 응답 종료 사유 (`stop`, `length`, `tool_calls` 등) |
| `logprobs`           | 단어별 예측 확률 정보 (`null`인 경우 생략) |

---

####  **2. `usage_metadata` 필드**
| **필드명**             | **설명** |
|-----------------------|---------|
| `input_tokens`       | 입력된 토큰 수 |
| `output_tokens`      | 모델이 생성한 출력 토큰 수 |
| `total_tokens`       | `input_tokens + output_tokens` |
| `input_token_details` | 입력 토큰의 세부 정보 (`audio`, `cache_read` 등) |
| `output_token_details`| 출력 토큰의 세부 정보 (`reasoning` 등) |

---

#### **3. `additional_kwargs` 필드**
| **필드명**      | **설명** |
|---------------|---------|
| `refusal`     | 모델이 응답을 거부한 경우 (`null`이면 정상 응답) |
| `custom_data` | 특정 애플리케이션에서 활용할 수 있는 추가 메타데이터 |

**추가 설명**  
- `additional_kwargs`는 **응답을 확장하는 메타데이터 저장소**입니다.  
- API 제한, 민감한 데이터 필터링 등 특정 정책에 따른 추가 정보를 포함할 수 있습니다.

---

#### 📌 **4. `tool_calls` 필드**
| **필드명**        | **설명** |
|-----------------|---------|
| `tool_calls`   | LLM이 실행할 도구 요청 정보 |
| `invalid_tool_calls` | 실행되지 않거나 실패한 도구 요청 목록 |

📌 **추가 설명**  
- `tool_calls`는 **LLM이 특정 도구 실행이 필요하다고 판단한 경우** 포함됩니다.  
- 하지만, `AIMessage` 자체가 도구를 실행하는 것은 아니며, LangChain의 `Runnable`이 이를 해석하여 실행합니다.  
- 예를 들어, LLM이 "날씨 정보를 조회해야 한다"고 판단하면, `tool_calls`에 `weather_api` 호출 정보가 포함됩니다.  
- 만약 해당 도구 호출이 실패하면 `invalid_tool_calls`에 기록됩니다.

**추후 `tool_calls`의 실제 동작 방식에 대해 상세히 다룰 예정입니다.**


In [4]:
print("\n📌 [JSON 포맷된 응답 - AIMessage]")
print(formatted_content)


📌 [JSON 포맷된 응답 - AIMessage]
{
  "content": "안녕하세요! 만나서 반가워요! 오늘 기분은 어때요? 😊",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 20,
      "prompt_tokens": 40,
      "total_tokens": 60,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-4o-mini-2024-07-18",
    "system_fingerprint": "fp_06737a9306",
    "finish_reason": "stop",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run-d30711d9-043e-47cd-aeb4-06408b0bde1b-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 40,
    "output_tokens": 20,
    "total_tokens": 60,
    "input_token_details": {
      "audio": 0,
      "cache_read

### 2. 대화 흐름 관리하기

여러 번의 대화를 이어가며 컨텍스트를 유지하는 방법을 살펴봅니다.

##### 먼저, LLM이 답변하기 어려운 질문을 해서, LLM이 이상한 답을 하는것을 확인한다

In [5]:
# 대화 예시 생성
conversation = [
    SystemMessage(content="당신은 심심풀이용 챗봇입니다. 상당히 밝고 친절합니다."),
    HumanMessage(content="퍼블릭에이아이는 어디에 위치한 회사야?")
]

# 이전 대화 내용을 포함하여 응답 생성
response = llm.invoke(conversation)
print(response.content)

퍼블릭에이아이(Public AI)는 미국에 본사를 두고 있는 인공지능 관련 회사입니다. 다양한 AI 솔루션과 서비스를 제공하고 있으며, 기술 혁신에 중점을 두고 활동하고 있습니다. 더 궁금한 점이 있으면 언제든지 물어보세요! 😊


##### LLM은 오늘 날짜 및 날씨를 모른다. 만약 Multiturn이라면?

In [6]:
# 이전 대화 내용을 포함한 메시지 리스트
conversation = [
    SystemMessage(content="당신은 심심풀이용 챗봇입니다. 상당히 밝고 친절합니다."),
    
    HumanMessage(content="안녕. 퍼블릭에이아이는 서울시 서초구 서초3동 1579-1 2층에 있는 회사야."), # Q1
    AIMessage(content="안녕하세요! 퍼블릭에이아이는 서초구에 있는 AI기업이군요 😊"), # A1
    
    HumanMessage(content="퍼블릭에이아이는 어디에 위치한 회사야?"), # 새로운 질문
]

# 이전 대화 내용을 포함하여 응답 생성
response = llm.invoke(conversation)

print("\n📝 [대화 내역]")
for msg in conversation:
    print(f"{msg.__class__.__name__}: {msg.content}")

print("\n🔹 [최종 응답]")
print(response.content)


📝 [대화 내역]
SystemMessage: 당신은 심심풀이용 챗봇입니다. 상당히 밝고 친절합니다.
HumanMessage: 안녕. 퍼블릭에이아이는 서울시 서초구 서초3동 1579-1 2층에 있는 회사야.
AIMessage: 안녕하세요! 퍼블릭에이아이는 서초구에 있는 AI기업이군요 😊
HumanMessage: 퍼블릭에이아이는 어디에 위치한 회사야?

🔹 [최종 응답]
퍼블릭에이아이는 서울시 서초구 서초3동 1579-1 2층에 위치해 있어요! 멋진 곳에 있네요! 궁금한 점이 더 있으면 언제든지 물어보세요! 😊


### 🛠 **멀티턴 대화 구성하기**

위 코드처럼 `HumanMessage`와 `AIMessage`를 누적하면 **이전 대화 내용을 유지하면서 멀티턴 대화를 구성**할 수 있습니다.  
> 물론, 위는 모든 기록을 남기게 되는것이므로, 대화가 길면 메모리 기반의 관리, 요약 기반의 관리 등 많은 방법들이 존재한다. 이는 추후 다루게 될 예정

### 3. ToolMessage 이해하기
- ToolMessage란?

In [9]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
import json
import yfinance as yf
from datetime import datetime

# 1. 오늘 날짜 조회 Tool
@tool
def tool_get_today_date() -> str:
    """오늘 날짜를 조회하여 반환합니다."""
    result = f"오늘 날짜는 {datetime.today().strftime('%Y-%m-%d')}입니다."
    print(f"[Tool 실행] tool_get_today_date -> {result}")
    return result

# 2. 범용 주가 조회 Tool
@tool
def tool_get_stock_price(ticker: str) -> str:
    """주어진 주식 심볼(Ticker)의 오늘 종가를 조회하여 반환합니다."""
    stock = yf.Ticker(ticker)
    today = datetime.today().strftime('%Y-%m-%d')
    data = stock.history(period="1d")
    
    if not data.empty:
        result = f"{today} 기준 {ticker} 종가는 {data['Close'].iloc[-1]:.2f} USD입니다."
    else:
        result = f"{ticker} 주가 정보를 가져올 수 없습니다."
    
    print(f"[Tool 실행] tool_get_stock_price({ticker}) -> {result}")
    return result

# 3. 서울 기온 조회 Tool
@tool
def tool_get_seoul_temperature() -> str:
    """서울의 현재 최고기온과 최저기온을 조회하여 반환합니다."""
    result = "현재 서울의 최고 기온은 5도, 최저 기온은 -2도입니다."
    print(f"[Tool 실행] tool_get_seoul_temperature -> {result}")
    return result

# Tools 리스트 생성 및 LLM 바인딩
tools = [tool_get_today_date, tool_get_stock_price, tool_get_seoul_temperature]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

# 메시지 흐름을 출력하는 함수
def print_messages(messages, step):
    print(f"\n[메시지 흐름: {step} 단계]")
    for i, msg in enumerate(messages):
        msg_type = msg.__class__.__name__
        print(f"{i + 1}. {msg_type}: {msg.content}")

"""
이곳부터, 실제 대화 흐름을 구성하는 코드입니다.
"""
# 사용자 질문 설정
query = "오늘 날짜는? 지금 서울날씨는?"
messages = [
    SystemMessage(content="당신은 유용한 도우미입니다."),
    HumanMessage(content=query)
]

# 초기 메시지 출력
print_messages(messages, "시작")

# 1단계: LLM이 응답을 생성 (tool_calls 확인)
response = llm.invoke(messages)
print("\n[1단계: LLM 응답 (tool_calls)]")
print(response)

# tool_calls 값 확인
if hasattr(response, "tool_calls"):
    print("\n[tool_calls 값 확인]")
    print(json.dumps(response.tool_calls, indent=2, ensure_ascii=False))

# 2단계: Tool 호출 수행 후, 새로운 메시지 추가
if response.tool_calls:
    tool_messages = []
    for call in response.tool_calls:
        tool_name = call["name"]
        tool_args = call["args"]
        tool_call_id = call["id"]  # tool_call_id 추가
        
        # ✅ Tool 실행 (invoke 방식 사용) - input 인자를 올바르게 전달
        tool_result = None
        if tool_name == "tool_get_today_date":
            tool_result = tool_get_today_date.invoke(input={})
        elif tool_name == "tool_get_stock_price":
            ticker = tool_args.get("ticker", "TSLA")  
            tool_result = tool_get_stock_price.invoke(input={"ticker": ticker})
        elif tool_name == "tool_get_seoul_temperature":
            tool_result = tool_get_seoul_temperature.invoke(input={})
        
        # ToolMessage 추가
        if tool_result:
            tool_messages.append(
                ToolMessage(content=tool_result, tool_call_id=tool_call_id)
            )

    # 2단계 이후 메시지 확인
    print_messages(tool_messages, "Tool 실행 후")

    # 3단계: LLM이 ToolMessage를 반영하여 최종 응답 생성
    messages.append(response)  # tool_calls가 있는 AIMessage 추가
    messages += tool_messages  # 실행된 ToolMessage 추가
    print_messages(messages, "ToolMessage 추가 후")

    final_response = llm.invoke(messages)
    
    print("\n[3단계: 최종 응답]")
    print(final_response)
    print_messages([final_response], "최종 응답")



[메시지 흐름: 시작 단계]
1. SystemMessage: 당신은 유용한 도우미입니다.
2. HumanMessage: 오늘 날짜는? 지금 서울날씨는?

[1단계: LLM 응답 (tool_calls)]
content='' additional_kwargs={'tool_calls': [{'id': 'call_JE3qspFUSFJNjiSj8C5SFtYs', 'function': {'arguments': '{}', 'name': 'tool_get_today_date'}, 'type': 'function'}, {'id': 'call_GMP27RnrkXhkOgt0LO6M0pJM', 'function': {'arguments': '{}', 'name': 'tool_get_seoul_temperature'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 124, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-748470d8-be63-44c2-bed5-caccd732a98f-0' tool_calls=[{'name': 'tool_get_today_date', 'args': {}, 'id': 'call_JE3qspFUSFJNj

- AIMessage가 tool_calls를 포함하고 있으면, LangChain은 AI가 직접 답변하지 않고 도구를 실행해야 한다고 판단.
- AIMessage는 즉시 ToolMessage를 생성하는 것이 아니라, 사용자가 tool_calls에 따라 해당 도구를 실행한 후 ToolMessage를 추가해야 함.
- ToolMessage를 추가한 후, 다시 AIMessage를 생성하면 이전의 ToolMessage를 보고 최종 응답을 생성.

In [11]:
1. 
AIMessage(content가 있고,tool_calls가 없다)

2. 
AIMessage(content="", tool_calls가 있다)

ToolMessage(날짜조회)

ToolMessage(주가조회)

AIMessage(content="")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3689821113.py, line 2)

### Message 메타데이터 활용하기

`Message`에 추가 정보를 포함하여 더 풍부한 대화를 구현할 수 있습니다.  
메타데이터를 활용하면 **대화 내역의 히스토리를 관리 및 추적**할 수 있으며, 어플리케이션 관점에서 다음과 같은 용도로 활용할 수 있습니다.

- **사용자 세션 관리**: `user_id`, `platform` 정보를 저장하여 사용자별 대화 흐름을 추적  
- **대화 분석 및 로깅**: `timestamp`, `version` 등의 데이터를 활용하여 대화 기록 분석  
- **컨텍스트 기반 응답**: 사용자 환경(웹/모바일 등)에 따라 다른 답변을 제공  

In [12]:
from datetime import datetime

# 메타데이터가 포함된 메시지 생성
messages_with_metadata = [
    SystemMessage(
        content="당신은 심심풀이용 챗봇입니다. 상당히 밝고 친절합니다.",
        additional_kwargs={
            "timestamp": datetime.now().isoformat(),
            "version": "1.0"
        }
    ),
    HumanMessage(
        content="안녕 만나서 반가워. 오늘 몇일이야?",
        additional_kwargs={
            "user_id": "user123",
            "platform": "web"
        }
    )
]

# 메시지 실행
response = llm.invoke(messages_with_metadata)

print("\n📝 [메타데이터가 포함된 메시지]")
for msg in messages_with_metadata:
    print(f"\n[{msg.__class__.__name__}]:")
    print(f"Content: {msg.content}")
    print(f"Metadata: {msg.additional_kwargs}")

print("\n🔹 [응답 메시지]")
print(response.content)


📝 [메타데이터가 포함된 메시지]

[SystemMessage]:
Content: 당신은 심심풀이용 챗봇입니다. 상당히 밝고 친절합니다.
Metadata: {'timestamp': '2025-03-07T15:34:34.450478', 'version': '1.0'}

[HumanMessage]:
Content: 안녕 만나서 반가워. 오늘 몇일이야?
Metadata: {'user_id': 'user123', 'platform': 'web'}

🔹 [응답 메시지]

